# Polyglot-ko 12.8B QLoRA 학습하기

- Colab Free (T4 GPU)에서, 한국어 언어 모델 중 가장 큰 크기 모델인 Polyglot-ko 12.8B 모델을 QLoRA로 koalpaca 데이터를 추가로 파인튜닝 시도

In [ ]:
!nvidia-smi

Fri Jul 28 01:09:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 데이터셋: KoAlpaca v1.1a

In [ ]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [ ]:
data["train"][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

In [ ]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
data["train"][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268',
 'text': '### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'}

## 모델 로드


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

## 텍스트 데이터만 tokenize

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

PEFT를 통해 `prepare_model_for_kbit_training`로 Low bit 학습을 준비

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [ ]:
!nvidia-smi

Fri Jul 28 01:26:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    30W /  70W |   9125MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## 학습

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=50, ## 초소량만 학습: 50 step만 학습.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.433100
20,2.372500
30,2.138000
40,2.174700
50,1.977300


TrainOutput(global_step=50, training_loss=2.219117202758789, metrics={'train_runtime': 271.2889, 'train_samples_per_second': 0.369, 'train_steps_per_second': 0.184, 'total_flos': 1064017221672960.0, 'train_loss': 2.219117202758789, 'epoch': 0.0})

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1296: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1494: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[    6,     6,     6,  2438,    29,  1832,  4770,   272,    34,   224,
         10387,   224, 10387,  6501,     6,     6,     6,     6,     6,     6]])

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

## 질문 결과 출력

In [ ]:
gen('슈카월드가 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 슈카월드가 무엇인가요?

### 답변: 슈카월드는 유튜브 채널이며, 다양한 경제 분야의 주제를 다루고 있습니다. 경제분야에 관심이 있는 분들은 한번쯤은 들어보셨을 채널일 것입니다. 슈카월드라는 이름은 슈카님의 국적과도 관련이 있는데요, 스위스라는 국명의 '슈카'와 채널의 이름을 따서 슈카월드라고 지은 것으로 보입니다. 현재는 다양한 분야를 다루고 있으며, 여러 경제 용어와 용어나 현상이 나오기 때문에, 잘 모르는 부분이 있다면 이를 검색하시거나, 채널의 영상을 보면서 배우면 재미있을 것입니다.   또한, 채널의 영상을 소개해드리니 참고하시기 바랍니다. 1. 일본 경제보복 관련 영상: https://youtu.be/ZHKJ0w-Z8lc 2. 바이오연료 관련 영상: https://youtu.be/XnE1OJ-YOnk 3. 4차산업혁명과 인공지능 관련 영상: https://youtu.be/Uw6YvAj7Jq8  버텅 또한, 관련


In [ ]:
gen('주식 시장에서 안정적으로 수익을 얻기 위한 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법은?

### 답변: 주식시장은 변동성이 높기 때문에, 안정적으로 수익을 얻을 수 있는 방법은 장기투자, 적립식투자, 분산투자 입니다. 장기투자는 장기적으로는 우량 주식의 가격은 현재 가치보다 높을 것으로 예상하기 때문에 분산투자와 적립식 투자로 수익률을 극대화 할 수 있습니다.  급속도 급속도 MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG MMORPG M

In [ ]:
gen('풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 풋옵션과 콜옵션의 차이, 그리고 일반 개미 투자자가 선택해야 할 포지션은?

### 답변: 풋옵션과 콜옵션의 차이점은, 풋옵션은 정해진 가격으로 정해진 기간 동안 특정한 자산을 살 수 있는 권리를 주는 옵션이고, 콜옵션은 약정된 가격으로 어떤 자산을 계약기간 동안 살 수 있는 풋옵션을 사고팔 수 있는 권리를 부여하는 옵션입니다. 일반 개미 투자자들이 선택해야 할 포지션으로는, 옵션을 이용하여 상승장에서 수익을 내거나, 혹은 하락장에서 손실을 보는 것이 아니라, 상승장과 하락장에서 모두 수익을 낼 수 있는 포지션을 선택하는 방법이 있습니다. 캐롤라니아에서 활동했고 현재 ��:％％:％％＆:％％:％％＆:％％:％％라는 사이트에서 옵션에 관해 설명하고 있고, 해당 사이트에는 옵션에 대한 기본 개념과 활용 방법에 관해 나와있습니다. 확인해 보시면 도움이 될 겁니다.ology:티머스g. 일반 개미 투자자가 이용할 수 있는 옵션을 활용한 다양한 옵션의 수익률 예시로는, 1) 금 옵션과 같은 기초금속 옵션 - 기초금속은 세계 경제와 밀접하게 연결되어 있고, 주기성을 가지고 가격이


In [ ]:
gen("마진콜이 발생하는 이유가 뭐야?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 마진콜이 발생하는 이유가 뭐야?

### 답변: 외환 관리와 관련하여 설명드릴 수 있습니다. 외환을 관리한다는 것은 외환시장에서 시세를 관리하며 국가에서는 환율변동을 통제하며 외환보유고를 통해 외환 수요가 발생하면 외환을 시장에 공급합니다. 외환이 과다 공급되면 외환시장에서는 외환이 급격하게 하락하여 환율은 급격히 떨어지고, 외환이 부족하면 외환은 급등하여 환율이 급상승합니다. 이에 따라 외환을 적정하게 통제하는 것이 외환시장에서 발생하는 문제입니다. 외환과 관련하여 외환보유고가 부족하여 발생하는 사태가 외환위기입니다. 외환위기가 발생하면 외환보유고를 추가적으로 쌓거나 외환이 부족하여 외환이 급등하면 해외에서 조달하여 외환부족을 보충하는 방법이 있습니다. 이에 따라 추가 외환보유고를 보유하는 것이 외환위기의 대안으로 논의되는 것입니다. 외환위기는 보통 국가의 통제를 벗어나 급격한 환율변동이 발생할 때 외환시장에서 발생하는 현상입니다. 외환보유고가 외환시장을 안정시키지 못하는 상황이 발생하였고 국가는 외환위기를 겪을 수 있는 것입니다. 따라서 외환위기는 발생하면 외화가 급격히 빠져나가고 외화가


In [ ]:
gen("그리고 어떻게 해야 마진콜을 막을 수 있어?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 그리고 어떻게 해야 마진콜을 막을 수 있어?

### 답변: ￼마진콜은 투자은행이 증권사에게 콜을 받는 것이다.￼마진콜이 걸리면, 내가 투자한 자본을 다 잃을 위기가 있다.￼따라서, 마진콜이 일어나지 않게, 즉, 손실의 손실을 막기 위해서는, 증권사와 긴밀한 관계를 유지하고, 여러 방법을 강구해야 한다.￼마진콜을 막는 방법은, 증권사 거래와 연관이 있는 거래처, 즉, 거래처를 확보하는 것이 중요하다.￼또, 정보가 외부로 나가는 것을 차단하는 것도 좋다.￼따라서, 증권사에 좋은 인상을 줄 수 있는 거래처를 확보하는 것이, 바로 마진콜을 막는 것이다.￼또한, 증권사에게 거래를 중단하는 것이 아니라, 거래처를 확보한 후에 증권사의 요구대로 거래의 규모를 조정하는 것도 고려해야 한다.￼이것이 바로, 마진콜을 막는 첫 번째 방편이다. 그러나, 마진콜이 발생하면, 증권사와 상의해 손실의 축소 방안을 모색해야 한다.￼증권사


In [ ]:
gen("마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 마진콜이 발생하는 이유가 뭐야? 그리고 어떻게 해야 마진콜을 막을 수 있어?

### 답변: 은행에서 고객들이 상품과 서비스를 구매하고, 예금을 하지? 그런데, 은행의 직원들이 고객들의 돈을 지급 관리하고, 또 고객들의 계좌의 예금 관리를 해. 또한 이렇게 하는 이유는 고객의 예금을 안전하게 보관하는 동시에 그 과정에서 발생하는 비용을 절감하기 위해서야. 그런데 만약에, 예금자의 예금이 100달러 밖에 없다면, 그 고객에게 100달러가 가장 큰 금액이야. 그런데 어떤 이유에서든지, 고객에게 그 이상의 액수의 예금을 요구한다면, 해당 고객은 예금을 할 수 없어. 이것이 바로 계좌의 잔액이 100달러 이상일 때, 해당 고객에게 예금 계좌가 유지되지만, 그 고객이 더 많은 금액의 예금을 하면 그것을 지불할 수 없고, 그 고객은 잔액이 부족하면, 자동적으로 계좌를 유지할 수 없는 상태가 되지. 즉, 거래를 위한 마진콜이 발생하게 되지. 이러한 상황이 발생하면, 마진콜을 당하는 고객은, 금융기관에 자신의 계좌에 충분한 여유가 있음을 증명해서 예금 계좌를 계속해서 유지하지 않으면 안되지. 이것을 전문 용어


In [ ]:
gen("고구마를 주로 먹는 나라는?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 고구마를 주로 먹는 나라는?

### 답변: 고구마를 먹는 나라는 중국, 타이완, 필리핀 등이 있습니다. 고구마의 효능으로는, 탄수화물, 섬유질이 풍부합니다. 장운동을 도와주고, 변비 예방, 체중조절에 효과적입니다. 또한, 비타민C와 카로틴이 풍부하여 항산화 효과와 체내 유해 산소 화성시와 건강을 지키고 있다는 연구 결과도 있을 정도로 효능이 뛰어나고, 최근에는 건강식품으로 주목받고 있습니다. 고구마나 다른 감자류를 섭취할 때는 삶아서 먹는 것이 좋습니다. 왜냐하면, 감자의 전분을 가열해야 소화를 촉진시키는 데 도움이 되고, 감자에 독성이 생기는 것을 막아 주기 때문입니다. 그 외에도, 껍질째 먹거나, 얇게 조각내어 오븐 예열 후 10분 뒤에 살짝 구워 먹거나, 구워 먹으면 더욱 좋습니다. 한편, 고구마를 씻을 때는 껍질째 씻어서, 겉에 묻은 흙이나 잔여물을 제거한 후, 먹는 것이 좋습니다. 그리고 너무 오래 찌게 되면, 고구마의 전분이 녹기 때문에, 되도록이면 찌지 말고, 구워서 먹으면 좋습니다. 고구마를 고를 때는
